In [1]:
import requests
import json
import forecastio
from geopy.geocoders import Nominatim
import pandas as pd
import import_ipynb
from Austin_airbnb import Austin_airbnb
import datetime as dt
from config import api_key

importing Jupyter notebook from Austin_airbnb.ipynb


Austin_airbnb.ipynb:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  {
Austin_airbnb.ipynb:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "cells": [
Austin_airbnb.ipynb:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "metadata": {},


In [2]:
geolocator = Nominatim(user_agent="ETL_Project")

location = geolocator.geocode("Austin, TX")

ATX_coord = []
ATX_coord.append(location.latitude)
ATX_coord.append(location.longitude)

ATX_coord

[30.2711286, -97.7436995]

In [3]:
forecast = forecastio.load_forecast(api_key, ATX_coord[0], ATX_coord[1])

byDay = forecast.daily()
print(byDay.summary)
print(byDay.icon)
print(byDay.data[4].temperatureHigh)

Rain on Wednesday and Thursday, with high temperatures falling to 54°F next Saturday.
rain
69.91


In [4]:
temp_high = []
temp_low = []
precip_perc = []
precip_type = []
precip_int = []
date = []

for i in byDay.data:
    date.append(i.time.date())
    temp_high.append(i.temperatureHigh)
    temp_low.append(i.temperatureLow)
    precip_perc.append(i.precipProbability)
    precip_int.append(i.precipIntensity)
    try:
        precip_type.append(i.precipType)
    except:
        precip_type.append("none")

date

dates = [dt.datetime.strftime(i, '%Y-%m-%d') for i in date]
dates

['2018-12-22',
 '2018-12-23',
 '2018-12-24',
 '2018-12-25',
 '2018-12-26',
 '2018-12-27',
 '2018-12-28',
 '2018-12-29']

In [5]:
df = pd.DataFrame({"Date": dates, 
                     "High Temp": temp_high, 
                     "Low Temp": temp_low, 
                     "Precipitation Probability": precip_perc,
                     "Precipitation Intensity": precip_int,
                     "Precipitation Type": precip_type
                    })
df.dtypes

Date                          object
High Temp                    float64
Low Temp                     float64
Precipitation Probability    float64
Precipitation Intensity      float64
Precipitation Type            object
dtype: object

In [6]:
ATX_table = pd.merge(df, Austin_airbnb, on="Date")
ATX_table["City"] = "Austin"
ATX_table

,Date,High Temp,Low Temp,Precipitation Probability,Precipitation Intensity,Precipitation Type,Under $50,$50-100,$100-150,$150-200,$200-250,City
0,2018-12-22,73.86,45.60,0.05,0.0002,rain,532,1220,1039,678,360,Austin
1,2018-12-23,59.55,39.66,0.07,0.0002,rain,615,1335,980,610,294,Austin
2,2018-12-24,57.69,54.30,0.07,0.0006,rain,592,1291,963,602,273,Austin
3,2018-12-25,71.67,64.01,0.27,0.0034,rain,584,1267,960,598,270,Austin
4,2018-12-26,69.91,55.11,0.84,0.0342,rain,607,1286,996,579,276,Austin
5,2018-12-27,68.30,46.53,0.58,0.0114,rain,624,1274,955,588,276,Austin
6,2018-12-28,60.69,38.78,0.00,0.0000,none,525,1071,877,684,360,Austin
7,2018-12-29,48.68,36.41,0.34,0.0012,rain,523,1044,858,651,380,Austin
